In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import datetime as dt
%matplotlib inline

### Read realtor data

In [2]:
realtor_2018_data = pd.read_csv(
    Path("realtor_data/2018_realtor_data.csv"))
realtor_2019_data = pd.read_csv(
    Path("realtor_data/2019_realtor_data.csv"))
display(realtor_2018_data)

,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size
0,2018-01-02,for_sale,450000.0,3.0,2.0,0.09,"15 William St, North Arlington, NJ, 07031",15 William St,North Arlington,New Jersey,7031.0,NaN
1,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
2,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
3,2018-01-02,for_sale,1245000.0,1.0,1.0,NaN,"57 Reade St Apt 5D, New York, NY, 10007",57 Reade St Apt 5D,New York,New York,10007.0,785.0
4,2018-01-02,for_sale,234900.0,3.0,2.0,0.87,"185 Gilsum St, Keene, NH, 03431",185 Gilsum St,Keene,New Hampshire,3431.0,1752.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25191,2018-12-31,for_sale,389000.0,1.0,1.0,NaN,"62 Park Ter W Apt A26, New York, NY, 10034",62 Park Ter W Apt A26,New York,New York,10034.0,800.0
25192,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,New York,10065.0,1936.0
25193,2018-12-31,for_sale,399000.0,NaN,NaN,1.15,"1 Mayer Ln, Pomona, NY, 10970",1 Mayer Ln,Pomona,New York,10970.0,NaN
25194,2018-12-31,for_sale,3475000.0,2.0,2.0,NaN,"200 Mercer St Apt 4E, New York City, NY, 10012",200 Mercer St Apt 4E,New York City,New York,10012.0,2500.0


In [3]:
csv_files = glob.glob('realtor_data/*')
csv_files

['realtor_data/2019_realtor_data.csv', 'realtor_data/2018_realtor_data.csv']

### Merge realtor data

In [4]:
realtor_data = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file)
            realtor_data = realtor_data.append(df_temp)
            
realtor_data.set_index("zip_code", drop=True, inplace=True)

display(realtor_data)

/var/folders/_p/41m0r9cd3bvb4zcy8kyv_4580000gn/T/ipykernel_2238/2874440782.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  realtor_data = realtor_data.append(df_temp)
/var/folders/_p/41m0r9cd3bvb4zcy8kyv_4580000gn/T/ipykernel_2238/2874440782.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  realtor_data = realtor_data.append(df_temp)


,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,state,house_size
zip_code,,,,,,,,,,,
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,New Jersey,NaN
7013.0,2019-01-02,for_sale,575000.0,4.0,3.0,0.13,"29 Village Rd, Clifton, NJ, 07013",29 Village Rd,Clifton,New Jersey,NaN
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,New Jersey,NaN
7603.0,2019-01-02,for_sale,435000.0,3.0,2.0,0.10,"192 Linwood Ave, Bogota, NJ, 07603",192 Linwood Ave,Bogota,New Jersey,NaN
1089.0,2019-01-02,for_sale,349900.0,4.0,2.0,0.24,"16 Dorwin Dr, West Springfield, MA, 01089",16 Dorwin Dr,West Springfield,Massachusetts,2002.0
...,...,...,...,...,...,...,...,...,...,...,...
10034.0,2018-12-31,for_sale,389000.0,1.0,1.0,NaN,"62 Park Ter W Apt A26, New York, NY, 10034",62 Park Ter W Apt A26,New York,New York,800.0
10065.0,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,New York,1936.0
10970.0,2018-12-31,for_sale,399000.0,NaN,NaN,1.15,"1 Mayer Ln, Pomona, NY, 10970",1 Mayer Ln,Pomona,New York,NaN


### Merge IRS data

In [8]:
csv_files = glob.glob('IRS_Income_tax_data/*.csv')
csv_files

['IRS_Income_tax_data/IRS_2019_data_sliced.csv',
 'IRS_Income_tax_data/IRS_2018_data_sliced.csv']

In [9]:
IRS_sliced_data = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file)
            IRS_sliced_data = IRS_sliced_data.append(df_temp, ignore_index=True)
            
display(IRS_sliced_data)

/var/folders/_p/41m0r9cd3bvb4zcy8kyv_4580000gn/T/ipykernel_2238/4150780073.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  IRS_sliced_data = IRS_sliced_data.append(df_temp, ignore_index=True)
/var/folders/_p/41m0r9cd3bvb4zcy8kyv_4580000gn/T/ipykernel_2238/4150780073.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  IRS_sliced_data = IRS_sliced_data.append(df_temp, ignore_index=True)


,STATE,zipcode,A18425
0,AL,0,2661.0
1,AL,0,22962.0
2,AL,0,54996.0
3,AL,0,56947.0
4,AL,0,292110.0
...,...,...,...
332089,WY,99999,0.0
332090,WY,99999,0.0
332091,WY,99999,0.0
332092,WY,99999,92.0


In [10]:
# Set the Index to zip-code
IRS_sliced_data.rename(columns = {'zipcode':'zip_code'}, inplace=True)
IRS_sliced_data.set_index('zip_code', drop=True, inplace=True)

display(IRS_sliced_data)

,STATE,A18425
zip_code,,
0,AL,2661.0
0,AL,22962.0
0,AL,54996.0
0,AL,56947.0
0,AL,292110.0
...,...,...
99999,WY,0.0
99999,WY,0.0
99999,WY,0.0


### Merge realtor and IRS data on zip_code (left join)

In [11]:
df1 = realtor_data.merge(IRS_sliced_data, how = 'left', on = "zip_code")
df1.drop(columns=["state"], inplace = True)

## The final DataFrame we are going to work with!!!

In [12]:
display(df1)

,sold_date,status,price,bed,bath,acre_lot,full_address,street,city,house_size,STATE,A18425
zip_code,,,,,,,,,,,,
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,NaN,NJ,123.0
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,NaN,NJ,540.0
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,NaN,NJ,1137.0
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,NaN,NJ,1640.0
7024.0,2019-01-02,for_sale,3480000.0,6.0,7.0,0.19,"116 Bluff Rd, Fort Lee, NJ, 07024",116 Bluff Rd,Fort Lee,NaN,NJ,8850.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10065.0,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,1936.0,NY,911.0
10065.0,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,1936.0,NY,1171.0
10065.0,2018-12-31,for_sale,3995000.0,2.0,3.0,NaN,"200 E 62nd St Apt 14E, New York, NY, 10065",200 E 62nd St Apt 14E,New York,1936.0,NY,1949.0
